# Constant pH Simulations in OpenMM

In this tutorial we will run a constant pH simulation of a protein at pH 7.5.  Two amino acids have pKas in this range: CYS (pKa 8.33) and HIS (two titratable hydrogens with pKa 6.5 and 7.1 respectively).  We therefore treat all CYS and HIS sidechains in the protein as variable groups.

We begin by importing all the packages we will need.

In [1]:
from openmm import *
from openmm.app import *
from openmm.unit import *
from constantph import ConstantPH
from reference_energy import ReferenceEnergyFinder

The first step is to define the force field and parameters for our simulation.  In fact, we need to define two different force fields: an explicit solvent force field to use for running the simulation, and an implicit solvent force field to use when deciding whether to change the protonation states of residues.  For this tutorial we will use the Amber ff14 force field and the GBn2 implicit solvent model.  We create the ForceField objects and define the parameters to be passed to `createSystem()`.

In [2]:
explicit_ff = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
implicit_ff = ForceField('amber14-all.xml', 'implicit/gbn2.xml')
explicit_params = dict(nonbondedMethod=PME, nonbondedCutoff=0.9*nanometers, constraints=HBonds)
implicit_params = dict(nonbondedMethod=CutoffNonPeriodic, nonbondedCutoff=2.0*nanometers, constraints=HBonds)

Next we define the integrator to use for running the simulation.  We also need to define a second integrator that will be used for relaxing solvent after an accepted Monte Carlo move.  The relaxation integrator uses a smaller time step (because there may be large forces at the start of relaxation) and a larger friction coefficient (to dissipate excess energy more quickly).

In [3]:
temperature = 300*kelvin
integrator = LangevinIntegrator(temperature, 1.0/picosecond, 0.004*picoseconds)
relaxation_integrator = LangevinIntegrator(temperature, 10.0/picosecond, 0.002*picosecond)

Before we can run the simulation, we need to determine reference energies for each of the relevant protonation states.  This involves simulating a model compound (in our case, a single capped amino acid in a box of water) at various pHs to identify what reference energy produces the correct pKa.  The reference energies are specific to the force field, solvation model, temperature, and other settings used in the simulation.  Any change to the settings requires recomputing them.

The ReferenceEnergyFinder class makes this very easy, and model compounds are provided for all the variable amino acids.  Let's start by loading the CYS model and creating a ConstantPH object.

In [4]:
pdb = PDBFile('model-compounds/CYS.pdb')
variants = {1: ['CYS', 'CYX']}
referenceEnergies = {1: [0.0, 0.0]}
cph = ConstantPH(pdb.topology, pdb.positions, 7.0, explicit_ff, implicit_ff, variants, referenceEnergies, 250,
                explicit_params, implicit_params, integrator, relaxation_integrator)

Let's consider the arguments to the ConstantPH constructor.  The first two are the model to simulate (the Topology and initial positions).  Next is the pH to run the simulation at.  In this case it doesn't matter what value we pass, because the ReferenceEnergyFinder will overwrite it to run simulations at various pHs.  Next we have the ForceFields, followed by a description of residues whose states will vary.  `1: ['CYS', 'CYX']` means that residue 1 has two possible states: `'CYS'` and `'CYX'`.  These values are passed to `Modeller.addHydrogens()` to tell it what variants to build Topologies for.

Next is the list of reference energies of the states.  It doesn't matter what values we specify at this point: ReferenceEnergyFinder will overwrite them with the correct values.

The next argument is the number of relaxation steps to integrate after an accepted Monte Carlo move.  We specify 250 steps, which means to relax the solvent for 0.5 ps, since the relaxation integrator has a step size of 0.002 picoseconds.

Finally we provide the parameters to pass to `createSystem()` and the integrators we created earlier.

Now to determine the reference energies.  We create a ReferenceEnergyFinder object.  The arguments are the ConstantPH object we just created, the experimental pKa we want to match, and the temperature the simulation will be run at.

In [5]:
finder = ReferenceEnergyFinder(cph, 8.33, temperature)

We call `findReferenceEnergies()` to run simulations at a variety of pHs and determine the reference energies.  We tell it to run for 20,000 iterations (that is, attempted Monte Carlo moves), and to take 20 time steps between attempted moves (80 fs, since the integrator's step size is 4 fs).  These are the default values, so we could have omitted them.  They are shown here just so you know what options are available.

On a good GPU, this should only take a few minutes to run.  You can increase the number of iterations to converge the energy more tightly.

In [6]:
finder.findReferenceEnergies(iterations=20000, substeps=20)
cys_ref_energies = cph.titrations[1].referenceEnergies
print(cys_ref_energies)

[0.0 kJ/mol, -331.9021734377429 kJ/mol]


Since the zero point of energy is arbitrary, it always assigns a reference energy of 0 to the first state.

HIS is slightly more complicated: it has two titratable hydrogens, each with a different pKa.  We therefore run ReferenceEnergyFinder twice to compute the energies of the two deprotonated states (HID and HIE) relative to the fully protonated state (HIP).  First for HID:

In [7]:
pdb = PDBFile('model-compounds/HIS.pdb')
variants = {1: ['HIP', 'HID']}
referenceEnergies = {1: [0.0, 0.0]}
cph = ConstantPH(pdb.topology, pdb.positions, 7.0, explicit_ff, implicit_ff, variants, referenceEnergies, 250,
                explicit_params, implicit_params, integrator, relaxation_integrator)
finder = ReferenceEnergyFinder(cph, 7.1, temperature)
finder.findReferenceEnergies()
hid_ref_energies = cph.titrations[1].referenceEnergies
print(hid_ref_energies)

[0.0 kJ/mol, -15.70777104670718 kJ/mol]


And now for HIE:

In [8]:
variants = {1: ['HIP', 'HIE']}
referenceEnergies = {1: [0.0, 0.0]}
cph = ConstantPH(pdb.topology, pdb.positions, 7.0, explicit_ff, implicit_ff, variants, referenceEnergies, 250,
                explicit_params, implicit_params, integrator, relaxation_integrator)
finder = ReferenceEnergyFinder(cph, 6.5, temperature)
finder.findReferenceEnergies()
hie_ref_energies = cph.titrations[1].referenceEnergies
print(hie_ref_energies)

[0.0 kJ/mol, -32.17455005951631 kJ/mol]


We are ready to simulate the protein.  First we load a file containing the starting structure.  The example shown here is the standard DHFR benchmark system.  You can use whatever protein you want.

In [9]:
pdb = PDBFile('dhfr.pdb')

We need to determine which residues to titrate, and record the variants and reference energies for each one.

In [10]:
variants = {}
referenceEnergies = {}
for residue in pdb.topology.residues():
    if residue.name == 'CYS':
        variants[residue.index] = ['CYS', 'CYX']
        referenceEnergies[residue.index] = cys_ref_energies
    if residue.name == 'HIS':
        variants[residue.index] = ['HIP', 'HID', 'HIE']
        referenceEnergies[residue.index] = [0.0*kilojoules_per_mole, hid_ref_energies[1], hie_ref_energies[1]]

Now we create the ConstantPH object for the real simulation.  We are interested in simulating the protein at pH 7.5, but results tend to converge faster if we use simulated tempering to let it explore a range of pH values, both lower and higher than the target.  For this example, we tell it to sample a range of pH values between 6.5 and 8.5.  The more titratable sites you have, the more closely spaced the values need to be.  If there are too few, the simulation may become stuck in a single pH and not be able to transition between them.

In [11]:
cph = ConstantPH(pdb.topology, pdb.positions, [6.5, 7.0, 7.5, 8.0, 8.5], explicit_ff, implicit_ff, variants, referenceEnergies, 250,
                explicit_params, implicit_params, integrator, relaxation_integrator)

Before running a simulation, it is always good to energy minimize the system.

In [12]:
cph.simulation.minimizeEnergy()

Next we need to equilibrate it.  This is important for two reasons.  In any simulation, you should always equilibrate to make sure your starting conformation is typical of thermal equilibrium.  In addition, the simulated tempering algorithm needs to determine weights of each of the pH values it explores.  It takes time for the weights to converge, and until they do, the simulation will not follow the correct distribution.

Running the equilibration simulation is very simple.  We call `step()` in the usual way to integrate time steps, alternating with calls to `attemptMCStep()` to consider changes to the pH and protonation states.

In [13]:
for i in range(2000):
    cph.simulation.step(50)
    cph.attemptMCStep(temperature)

Running the production simulation is the same, but it is essential to record the current pH any time we save any kind of output.  If we want to study the protein's behavior at pH 7.5, we should ignore any states that correspond to a different pH.

Depending on the application, we may also want to record the states of the titratable residues.  In this case we will simply print out all of these values.  That is enough to show how to retrieve them.  In your own simulations, you can use them however you want.

In [14]:
for i in range(20):
    cph.simulation.step(50)
    cph.attemptMCStep(temperature)
    print(cph.pH[cph.currentPHIndex], [variants[index][cph.titrations[index].currentIndex] for index in variants])

7.0 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HIP', 'CYS']
7.5 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HIP', 'CYS']
8.0 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HID', 'CYS']
8.5 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HID', 'CYS']
8.5 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HID', 'CYS']
8.5 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HID', 'CYS']
7.5 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HIP', 'CYS']
7.0 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HIP', 'CYS']
8.0 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HIP', 'CYS']
7.0 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HIP', 'CYS']
6.5 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HIP', 'CYS']
8.0 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HIP', 'CYS']
7.0 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HIP', 'CYS']
8.5 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HID', 'CYS']
8.0 ['HIP', 'CYS', 'HIP', 'HID', 'HID', 'HIP', 'CYS']
8.0 ['HIP', 'CYS', 'HIP', 'HID', 'HID', 'HIP', 'CYS']
7.5 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HIP', 'CYS']
7.5 ['HIP', 'CYS', 'HIP', 'HIP', 'HID', 'HIP', 'CYS']
7.0 ['HIP', 'CYS', 'HIP', 'H

The local environment can cause the pKa of a residue to be considerably different from that of the isolated model compound.  We see that different residues spend most of their time in different states, with occasional fluctuations.